[Milvus test](https://blog.csdn.net/Andrwin/article/details/124324248)

In [60]:
import random
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

### 1 链接milvus服务器

In [4]:
connections.connect(host='3.68.74.215', port='19530')

### 2 建表

In [6]:
collection_name = "Feat_test"# 表名
dim = 2 # 向量，dim=2代表向量只有两列，自己的数据的话一个向量有多少个元素就多少列

In [22]:
id = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, )# 主键索引
features = FieldSchema(name="features", dtype=DataType.FLOAT_VECTOR, dim=dim)
schema = CollectionSchema(fields=[id, features], description="Save features")# 描述

In [23]:
collection = Collection(name=collection_name, schema=schema, using='default', shards_num=2,consistency_level="Strong")# 建表

### 3 读表，并查看信息

In [30]:
collection = Collection("Feat_test")  # Get an existing collection.
print(collection.schema)                # Return the schema.CollectionSchema of the collection.

{'auto_id': False, 'description': 'Save features', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'features', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 2}}]}


In [31]:
print(collection.description)           # Return the description of the collection.

Save features


In [32]:
print(collection.name     )             # Return the name of the collection.

Feat_test


In [33]:
print(collection.is_empty )             # Return the boolean value that indicates if the collection is empty.

True


In [34]:
print(collection.num_entities )         # Return the number of entities in the collection.

0


In [35]:
print(collection.primary_field )        # Return the schema.FieldSchema of the primary key field.

{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}


In [36]:
print(collection.partitions)            # Return the list[Partition] object.

[{"name":"_default","collection_name":"Feat_test","description":""}]


In [37]:
print(collection.indexes )              # Return the list[Index] object.

[]


### 4 插入数据，建立索引

In [38]:
import random
data = [[i for i in range(2000)],
        [[random.random() for _ in range(2)] for _ in range(2000)],]

In [47]:
collection = Collection("Feat_test")      # Get an existing collection.
mr = collection.insert(data)
print(mr)

(insert count: 2000, delete count: 0, upsert count: 0, timestamp: 444023530107699206, success count: 2000, err count: 0)


In [48]:
index_params = {"metric_type":"L2","index_type":"IVF_FLAT","params":{"nlist":1024}}
collection = Collection("Feat_test")      # Get an existing collection.

In [49]:
collection.create_index(
    field_name="features", 
    index_params=index_params
    )

alloc_timestamp unimplemented, ignore it


Status(code=0, message=)

### 5 检索向量

In [50]:
collection = Collection("Feat_test")      # Get an existing collection.
collection.load()

In [52]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

In [53]:
results = collection.search(
    data=[[0.1, 0.2]], 
    anns_field="features", 
    param=search_params, 
    limit=10, 
    expr=None,
    consistency_level="Strong"
)

In [54]:
results[0].ids

[999, 1001, 1475, 75, 358, 25, 208, 918, 1353, 1484]

In [55]:
results[0].distances

[0.0003853969683405012, 0.0004526191914919764, 0.0005387334385886788, 0.000669920293148607, 0.0007647850434295833, 0.001225667423568666, 0.0014360530767589808, 0.0015708350110799074, 0.0015768151497468352, 0.0020566426683217287]

In [56]:
data[1][204]

[0.2039014881907596, 0.6381399172221718]

### 6 释放内存

In [57]:
collection.release()